## Import library

In [23]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn

## Import Env File

In [2]:
from dotenv import load_dotenv  

load_dotenv()
datasets_path = os.getenv('DATASET_PATH')
models_path = os.getenv('MODELS_PATH')

## Device Configuration

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Parameters

In [8]:
num_classes = 2
batch_size = 16
num_epochs = 20
learning_rate = 0.001
test_split_ratio = 0.2
image_size = (224, 224)

## Data Preprocessing

In [49]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Custom Dataset With GLCM Features

In [ ]:
class GLCMCNNHybridDataset(Dataset):
    def __init__(self, image_folder_dataset, transform=None):
        self.image_folder_dataset = image_folder_dataset
        self.transform = transform

    def extract_glcm_features(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        features = [
            graycoprops(glcm, 'contrast')[0, 0],
            graycoprops(glcm, 'dissimilarity')[0, 0],
            graycoprops(glcm, 'homogeneity')[0, 0],
            graycoprops(glcm, 'energy')[0, 0],
            graycoprops(glcm, 'correlation')[0, 0],
        ]
        return features

    def __getitem__(self, index):
        image, label = self.image_folder_dataset[index]

        # Terapkan transformasi langsung pada objek PIL.Image
        if self.transform:
            image = self.transform(image)  # Transformasi menghasilkan tensor

        # Konversi ke numpy untuk ekstraksi GLCM
        image_np = np.array(image)
        glcm_features = self.extract_glcm_features(image_np)
        glcm_features = torch.tensor(glcm_features, dtype=torch.float32)

        return image, glcm_features, label  # Mengembalikan tensor gambar

    def __len__(self):
        return len(self.image_folder_dataset)

## Data Loaders

In [51]:
# Buat dataset dengan ImageFolder
base_dataset = datasets.ImageFolder(root=datasets_path, transform=transform)

# Bungkus dengan GLCMCNNHybridDataset
hybrid_dataset = GLCMCNNHybridDataset(base_dataset, transform=transform)

# Split dataset menjadi training dan testing
test_size = int(test_split_ratio * len(hybrid_dataset))
train_size = len(hybrid_dataset) - test_size
train_dataset, test_dataset = random_split(hybrid_dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Load Pre-Trained Model (EfficientNet)

In [39]:
# Load EfficientNet-B0
weights = EfficientNet_B0_Weights.DEFAULT
cnn_model = efficientnet_b0(weights=weights)
cnn_feature_size = cnn_model.classifier[1].in_features  # EfficientNet features

# Replace classification layer to get features
cnn_model.classifier = nn.Identity()
cnn_model = cnn_model.to(device)

## Hybrid Model

In [14]:
class HybridModel(nn.Module):
    def __init__(self, cnn_model, glcm_feature_size, cnn_feature_size, num_classes):
        super(HybridModel, self).__init__()
        self.cnn_model = cnn_model
        self.fc1 = nn.Linear(glcm_feature_size + cnn_feature_size, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, image, glcm_features):
        cnn_features = self.cnn_model(image)
        combined_features = torch.cat((glcm_features, cnn_features), dim=1)
        x = self.fc1(combined_features)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## Initialize Hybrid Model

In [18]:
glcm_feature_size = 5  # Number of GLCM features
model = HybridModel(cnn_model, glcm_feature_size, cnn_feature_size, num_classes).to(device)

## Train The Model

In [52]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, glcm_features, labels in train_loader:
        images, glcm_features, labels = images.to(device), glcm_features.to(device), labels.to(device)

        # Forward pass
        outputs = model(images, glcm_features)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

## Evaulation

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, glcm_features, labels in test_loader:
        images, glcm_features, labels = images.to(device), glcm_features.to(device), labels.to(device)
        outputs = model(images, glcm_features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

## Save The Model

In [ ]:
torch.save(model.state_dict(), 'hybrid_glcm_cnn_model.pth')